In [1]:
from __future__ import print_function, division
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt
import math
import numpy as np
import wx
import os
import cv2
import pywt
from pylab import imread
from PIL import Image, ImageDraw
import pywt
from pywt import dwt2, idwt2

In [2]:
# Load Images
def get_path(wildcard):
    app = wx.App(None)
    style = (wx.FD_OPEN | wx.FD_FILE_MUST_EXIST) | wx.FD_MULTIPLE
    dialog = wx.FileDialog(None, 'Open', wildcard=wildcard, style=style)
    if dialog.ShowModal() == wx.ID_OK:
        path = dialog.GetPaths()
    else:
        path = None
    dialog.Destroy()
    return path

# Init lists of Image, Name
listImage = []
listName = []
i = 0
for link in get_path('*.*'):
    name = link
    name = os.path.basename(link)
    if (os.path.splitext(name)[1]=='.jpg') or (os.path.splitext(name)[1]=='.png'):
        name = os.path.splitext(name)[0]
        image = imread(link)
        listName.append(name)
        listImage.append(image)

# Compress
def Compress(image):
    if len(image.shape)>2:
        Red_Image = image[:,:,0];
        Green_Image = image[:,:,1];
        Blue_Image = image[:,:,2];
        LL, (LH, HL, HH) = dwt2(Red_Image,'db1')
        A = idwt2((LL, (None, None, None)),'db1')
        LL, (LH, HL, HH) = dwt2(Green_Image,'db1')
        B = idwt2((LL, (None, None, None)),'db1')
        LL, (LH, HL, HH) = dwt2(Blue_Image,'db1')
        C = idwt2((LL, (None, None, None)),'db1')
        New_Image = cv2.merge((A,B,C))
        img = Image.fromarray(New_Image.astype('uint8'), 'RGB')
        ImageDraw.Draw(img)
        img.save('out.png')
        size2 = os.stat('out.png').st_size
        os.remove('out.png')
    else:
        LL, (LH, HL, HH) = dwt2(image,'db1')
        New_Image = idwt2((LL, (None, None, None)),'db1')
        plt.imsave('out.png', New_Image, cmap='gray')
        size2 = os.stat('out.png').st_size
    image = New_Image
    return New_Image, size2
    
def Solve(image,levels):
    if len(image.shape)>2:
        img = Image.fromarray(image.astype('uint8'), 'RGB')
        ImageDraw.Draw(img)
        img.save('out.png')
        size1 = os.stat('out.png').st_size 
    else:
        plt.imsave('out.png', image, cmap='gray')
        size1 = os.stat('out.png').st_size
        
    level = levels
    while level>0:
        image,size2 = Compress(image)
        level = level - 1
    if len(image.shape)>2:
        img = Image.fromarray(image.astype('uint8'), 'RGB')
        display(img)
    else:
        plt.imshow(image,cmap='gray')
        plt.show()
    ratio = math.ceil(size1/size2)
    print(ratio)
# Choose Image
def ChosenImage(image_name):
    for i in range(0,len(listImage)):
        if listName[i]==image_name:
            image_number = i
            image = listImage[i]
            width = image.shape[1]
    interact(Solve, image=fixed(image), levels = widgets.FloatSlider(min=1, max = pywt.dwtn_max_level((image.shape[1],image.shape[0]), 'db1'), step=1, value=1))
        
interact(ChosenImage,image_name=listName)

interactive(children=(Dropdown(description='image_name', options=('grayscale', '78633293_2577918115761742_6671…

<function __main__.ChosenImage(image_name)>